# Getting Started with Sentiment Analysis Pipeline in caikit

## Setup

In [1]:
# %pip install caikit transformers requests
# mamba install pytorch cpuonly -c pytorch
# mamba install grpcio

In [2]:
# %%bash 
# pip install caikit[runtime-grpc] -qqq
# pip install caikit[runtime-http] -qqq

In [3]:
!python --version

Python 3.9.18


In [4]:
%pip show protobuf

Name: protobuf
Version: 4.24.4
Summary: 
Home-page: https://developers.google.com/protocol-buffers/
Author: protobuf@googlegroups.com
Author-email: protobuf@googlegroups.com
License: 3-Clause BSD License
Location: /home/msivanes/miniconda3/envs/fastchai/lib/python3.9/site-packages
Requires: 
Required-by: caikit, grpcio-health-checking, grpcio-reflection, py-to-proto
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip show py-to-proto

Name: py-to-proto
Version: 0.5.1
Summary: A tool to dynamically create protobuf message classes from python data schemas
Home-page: https://github.com/IBM/py-to-proto
Author: Gabe Goodhart
Author-email: gabe.l.hart@gmail.com
License: MIT
Location: /home/msivanes/miniconda3/envs/fastchai/lib/python3.9/site-packages
Requires: alchemy-logging, protobuf
Required-by: caikit
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip show grpcio

Name: grpcio
Version: 1.59.0
Summary: HTTP/2-based RPC framework
Home-page: https://grpc.io
Author: The gRPC Authors
Author-email: grpc-io@googlegroups.com
License: Apache License 2.0
Location: /home/msivanes/miniconda3/envs/fastchai/lib/python3.9/site-packages
Requires: 
Required-by: caikit, grpcio-health-checking, grpcio-reflection, py-grpc-prometheus
Note: you may need to restart the kernel to use updated packages.


## Outline

- Data Module
- Module
- config
- Runtime
- Client

## Text Classification Example

In [7]:
from fastcore.all import *
import warnings; warnings.filterwarnings('ignore')

/home/msivanes/.local/lib/python3.9/site-packages/fastprogress/fastprogress.py:102: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


### Requirements

In [8]:
%%writefile requirements-caikit.txt

caikit[runtime-grpc, runtime-http]

# Only needed for HuggingFace
scipy
# torch
# transformers~=4.27.2

# For http client
requests

Overwriting requirements-caikit.txt


### Data Module

In [9]:
Path('./text_sentiment/data_model').mkdir(exist_ok=True, parents=True)

In [10]:
%%writefile ./text_sentiment/data_model/classification.py

from typing import List
from caikit.core import DataObjectBase

from caikit.core.data_model import dataobject

# A DataObject is a data model class that is backed by a @dataclass. 
@dataobject(package="text_sentiment.data_model")
class ClassInfo(DataObjectBase):
    class_name: str
    conf: float
@dataobject(package="text_sentiment.data_model")
class ClassificationPrediction(DataObjectBase):
    classes: List[ClassInfo]

Overwriting ./text_sentiment/data_model/classification.py


In [11]:
%%writefile ./text_sentiment/data_model/__init__.py

from .classification import ClassificationPrediction

Overwriting ./text_sentiment/data_model/__init__.py


### Runtime Model

In [12]:
Path('./text_sentiment/runtime_model').mkdir(exist_ok=True, parents=True)

In [13]:
%%writefile ./text_sentiment/runtime_model/hf_module.py

from caikit.core import ModuleBase, ModuleLoader, ModuleSaver, TaskBase, task, module
from text_sentiment.data_model.classification import ClassificationPrediction, ClassInfo
from transformers import pipeline

@task(required_parameters={"text_input": str},output_type=ClassificationPrediction)
class HFSentimentTask(TaskBase): pass # defines input args and output type for task

@module('8f72161-c0e4-49b0-8fd0-7587b3017a35', 'HFSentimentModule', '0.0.1', HFSentimentTask)
class HFSentimentModule(ModuleBase): # inherits from ModuleBase and wraps the sentiment analysis pipeline from HF
    def __init__(self, model_path) -> None:
        super().__init__()
        loader = ModuleLoader(model_path) # loads the model from the path
        config = loader.config # gets the config from the model
        model = pipeline(model=config.hf_artifact_path, task='sentiment-analysis')
        self.sentiment_pipeline = model # sets the pipeline as an attribute of the module
        
    def run(self, text_input: str)->ClassificationPrediction:
        raw_results = self.sentiment_pipeline([text_input]) # runs the pipeline on the input text
        class_info = []
        for result in raw_results: 
            class_info.append(ClassInfo(class_name=result['label'], conf=result['score'])) # creates a ClassInfo object for each result
        return ClassificationPrediction(classes=class_info) # returns a ClassificationPrediction object
    
    @classmethod
    def bootstrap(cls, model_path='distilbert-base-uncased-finetuned-sst-2-english'): # classmethod to load a HF based caikit model
        return cls(model_path=model_path)
    
    def save(self, model_path, **kwargs):
        import os
        module_saver = ModuleSaver(self, model_path=model_path) # saving modules and context manager for cleaning up after saving
        with module_saver:
            rel_path, _ = module_saver.add_dir("hf_model")
            save_path = os.path.join(model_path, rel_path)
            self.sentiment_pipeline.save_pretrained(save_path)
            module_saver.update_config({"hf_artifact_path": rel_path})
    
    @classmethod
    def load(cls, model_path): # classmethod to load a HF based caikit model
        return cls(model_path=model_path)

Overwriting ./text_sentiment/runtime_model/hf_module.py


In [14]:
%%writefile ./text_sentiment/runtime_model/__init__.py

from .hf_module import HFSentimentModule

Overwriting ./text_sentiment/runtime_model/__init__.py


### Config

In [15]:
%%writefile ./text_sentiment/config.yml

runtime:
    library: text_sentiment

Overwriting ./text_sentiment/config.yml


In [16]:
Path('./models/text_sentiment').mkdir(exist_ok=True, parents=True)

In [17]:
%%writefile ./models/text_sentiment/config.yml

module_id: 8f72161-c0e4-49b0-8fd0-7587b3017a35
name: HFSentimentModule
version: 0.0.1

Overwriting ./models/text_sentiment/config.yml


### Runtime


In [18]:
# Kill the process using a particular port
# !lsof -ti tcp:8086 | xargs kill -9

In [19]:
# !lsof -ti tcp:8086

In [20]:
%%writefile start_runtime.py

from os import path
import sys
import alog
from caikit.runtime.__main__ import main
import caikit

if __name__ == "__main__":
    models_directory = path.abspath(path.join(path.dirname(__file__), "models"))
    # models_directory = path.abspath(path.join(path.dirname('.'), "models"))
    caikit.config.configure(config_dict=dict(
        merge_strategy="merge", runtime=dict(
            local_models_dir=models_directory, library="text_sentiment", grpc=dict(enabled=True), http=dict(enabled=True)
        )
    ))
    sys.path.append(path.abspath(path.join(path.dirname(__file__), "../")))
    alog.configure(default_level="debug")
    main()

Overwriting start_runtime.py


In [21]:
%%writefile ./text_sentiment/__init__.py

from os import path
from . import data_model, runtime_model
import caikit

CONFIG_PATH = path.realpath(path.join(path.dirname(__file__), "config.yml"))
caikit.configure(CONFIG_PATH)

Overwriting ./text_sentiment/__init__.py


### Client

In [22]:
%%writefile ./client.py

from caikit.config.config import get_config # interacts with config.yml
from caikit.runtime import get_inference_request # return the inference request DataModel for the Module or Task Class 
from caikit.runtime.service_factory import ServicePackageFactory
from text_sentiment.runtime_model.hf_module import HFSentimentModule
import caikit, grpc, requests, json

if __name__ == "__main__":
    caikit.config.configure(
        config_dict=dict(merge_strategy='merge',
                        runtime=dict(library='text_sentiment', grpc=dict(enabled=True), http=dict(enabled=True)),)
    )
    inference_service = ServicePackageFactory.get_service_package(
        ServicePackageFactory.ServiceType.INFERENCE
    ) # ServicePackage: A container with properties referencing everything you need to bind a concrete Servicer implementation to a protobufs Service and grpc Server

    model_id = 'text_sentiment'

    if get_config().runtime.grpc.enabled:
        # setup grpc client
        port = 8085
        channel= grpc.insecure_channel(f'localhost:{port}')
        client_stub = inference_service.stub_class(channel)
        
        for text in ['I am not feeling well today', 'Today is a nice sunny day']:
            request = get_inference_request(task_or_module_class=HFSentimentModule.TASK_CLASS)(text_input=text).to_proto()
            response = client_stub.HFSentimentTaskPredict(request, 
                                                        metadata=[('mm-model-id', model_id)],
                                                        timeout=1)
            print('Text: ', text)
            print('Response from gRPC: ', response)
            
    if get_config().runtime.http.enabled:
        port = 8080
        for text in ['I am not feeling well today', 'Today is a nice sunny day']:
            payload = {"inputs": text}
            response = requests.post(
                f"http://localhost:{port}/api/v1/{model_id}/task/hugging-face-sentiment",
                json=payload,
                timeout=1,
            )
            print("\nText:", text)
            
            print("RESPONSE from HTTP:", json.dumps(response.json(), indent=4))

Overwriting ./client.py


In [23]:
# Install the dependencies
# %pip install -r requirements-caikit.txt

In [24]:
# Running the caikit runtime
!python start_runtime.py

2023-10-11T19:31:54.001360 [RUNTI:DBUG] Starting up caikit.runtime.grpc_server
{"channel": "COM-LIB-INIT", "exception": null, "level": "info", "log_code": "<RUN11997772I>", "message": "Loading service module: text_sentiment", "num_indent": 0, "thread_id": 139675964474624, "timestamp": "2023-10-11T19:31:54.002445"}


{"channel": "COM-LIB-INIT", "exception": null, "level": "info", "log_code": "<RUN11997772I>", "message": "Loading service module: caikit.interfaces.common", "num_indent": 0, "thread_id": 139675964474624, "timestamp": "2023-10-11T19:31:56.146893"}
{"channel": "COM-LIB-INIT", "exception": null, "level": "info", "log_code": "<RUN11997772I>", "message": "Loading service module: caikit.interfaces.runtime", "num_indent": 0, "thread_id": 139675964474624, "timestamp": "2023-10-11T19:31:56.147143"}
{"channel": "COM-LIB-INIT", "exception": null, "level": "info", "log_code": "<RUN11997772I>", "message": "Loading service module: text_sentiment", "num_indent": 0, "thread_id": 139675964474624, "timestamp": "2023-10-11T19:31:56.150158"}
{"channel": "COM-LIB-INIT", "exception": null, "level": "info", "log_code": "<RUN11997772I>", "message": "Loading service module: caikit.interfaces.common", "num_indent": 0, "thread_id": 139675964474624, "timestamp": "2023-10-11T19:31:56.150415"}
{"channel": "COM-LIB-

## fin

In [27]:
from caikit.config.config import get_config
get_config()

{'config_files': '',
 'merge_strategy': 'merge',
 'enable_error_checks': True,
 'max_exception_log_messages': 4,
 'model_management': {'trainers': {'default': {'type': 'LOCAL',
    'import_class': None,
    'config': {'retention_duration': '1d',
     'use_subprocess': False,
     'subprocess_start_method': 'spawn'}}},
  'finders': {'default': {'type': 'LOCAL', 'config': {'load_path': None}}},
  'initializers': {'default': {'type': 'LOCAL',
    'config': {'backend_priority': [{'type': 'LOCAL'}]}}}},
 'log': {'level': 'info',
  'filters': 'botocore:off,urllib3:off,matplotlib:off,boto3:off,jnius.reflect:off',
  'channel_width': 12,
  'thread_id': True,
  'formatter': 'json'},
 'data_streams': {'file_source_base': None},
 'runtime': {'library': 'sample_lib',
  'local_models_dir': 'models',
  'lazy_load_local_models': False,
  'lazy_load_poll_period_seconds': 10,
  'lazy_load_retries': 2,
  'load_threads': None,
  'tls': {'server': {'key': '', 'cert': ''}, 'client': {'cert': ''}},
  'grpc':